In [36]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [37]:
#tf.enable_eager_execution()

In [38]:
#!conda install -q tensorflow-datasets

In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [4]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [23]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [24]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [27]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 3s 130us/sample - loss: 0.5644 - accuracy: 0.7502 - val_loss: 0.4018 - val_accuracy: 0.8351
Epoch 2/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.3427 - accuracy: 0.8578 - val_loss: 0.3378 - val_accuracy: 0.8543
Epoch 3/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.2791 - accuracy: 0.8849 - val_loss: 0.3278 - val_accuracy: 0.8593
Epoch 4/10
25000/25000 [==============================] - 2s 95us/sample - loss: 0.2418 - accuracy: 0.9058 - val_loss: 0.3378 - val_accuracy: 0.8565
Epoch 5/10
25000/25000 [==============================] - 2s 96us/sample - loss: 0.2160 - accuracy: 0.9185 - val_loss: 0.3545 - val_accuracy: 0.8510
Epoch 6/10
25000/25000 [==============================] - 2s 98us/sample - loss: 0.1958 - accuracy: 0.9279 - val_loss: 0.3753 - val_accuracy: 0.8463
Epoch 7/10
25000/25000 [==============================]

In [30]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [31]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b'i have no idea what the other reviewer is talking about this was a wonderful movie and created a sense of the era that feels like time travel the characters are truly young mary is a strong match for byron claire is juvenile and a tad annoying <OOV> is a convincing beaten down <OOV> all are beautiful curious and <OOV> not the frightening <OOV> they are in gothic br br gothic works as an independent piece of shock film and i loved it for different reasons but this works like a <OOV> and <OOV> film and was from my <OOV> the best capture of what the summer must have felt like romantic yes but completely <OOV> my interest in the
b'I have no idea what the other reviewer is talking about- this was a wonderful movie, and created a sense of the era that feels like time travel. The characters are truly young, Mary is a strong match for Byron, Claire is juvenile and a tad annoying, Polidori is a convincing beaten-down sycophant... all are beautiful, curious, and decadent... not the frightening

In [33]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()